# TP3 - Probabilidad multidimensional

## Análisis estadístico de datos, 2021

## Alvaro Concha

In [1]:
"""Instalar librerias necesarias"""
%pip install numpy seaborn

Note: you may need to restart the kernel to use updated packages.


---
## Ejercicio 7 (Para entregar)
Simular una variable aleatoria $X = (X_1, X_2)$ que
sigue una distribución binormal con parámetros $\mu_1 = 2.3$, $\mu_2 = 1.5$, $\sigma_1 = 1.2$, $\sigma_2 = 0.5$ y correlación $\rho = 0.7$.

Usar que la distribución conjunta $f(x_1, x_2) = h_2(x_2|x_1) g_1(x_1)$, con $g_1(x_1)$ la distribución marginal de $X_1$ y $h_2(x_2|x_1)$ la distribución condicional de $X_2$ dado $X_1$.

Esta distribución condicional se puede expresar como $h_2(x_2|x_1) = \mathcal{N}(\mu_2^\prime, \sigma_2^\prime)$ con $\mu_2^\prime = \mu_2 + \rho \frac{\sigma_2}{\sigma_1} (x_1 - \mu_1)$ y ${\sigma_2^\prime}^2 = \sigma_2^2 (1 - \rho^2)$.

Repetir la simulación 1000 veces.

Calcular la fracción de eventos caen en la elipse $1\sigma$ y comparar con la probabilidad contenida dentro de dicha región.

Graficar los datos simulados junto a la elipse $1\sigma$.

*Nota: un punto $(x_1, x_2)$ pertenece a la elipse $1\sigma$ si la forma cuadrática asociada $q(x_1, x_2) \le 1$.*

## Solución
Sea la variable aleatoria binormal
$$
X = (X_1, X_2) \sim \mathcal{N}_2(\boldsymbol{\mu}_X, \Sigma_X),
$$
con media
$$
\begin{align*}
\boldsymbol{\mu}_X &= (\mu_1, \mu_2)\\
&= (2.3, 1.5),
\end{align*}
$$
y matriz de covarianza
$$
\Sigma_X = \begin{pmatrix}
\sigma_1^2 & \rho\sigma_1\sigma_2\\
\rho\sigma_1\sigma_2 & \sigma_2^2
\end{pmatrix},
$$
con $\sigma_1 = 1.2$, $\sigma_2 = 0.5$ y $\rho = 0.7$.

* Implementamos estos parámetros en el código como `mu1`, `mu2`, `sigma1`, `sigma2` y `rho`.
* Calculamos el vector de miedias `mu_X = get_mu_X(mu1=mu1, mu2=mu2)` y la matriz de covarianza `Sigma_X = get_Sigma_X(sigma1=sigma1, sigma2=sigma2, rho=rho)`.

Entonces, podemos simular la distribución de probabilidad conjunta
$$
\begin{align*}
P(X = (x_1, x_2)) &= f(x_1, x_2)\\
&= h_2(x_2|x_1) g_1(x_1),
\end{align*}
$$
generando $X_1$ y $X_2$ por seaparado, con probabilidad marginal
$$
\begin{align*}
P(X_1 = x_1) &= g_1(x_1)\\
&= \mathcal{N}[\mu_1, \sigma_1](x_1),
\end{align*}
$$
y probabilidad condicional
$$
\begin{align*}
P(X_2 = x_2 | X_1 = x_1) &= h_2(x_2|x_1)\\
&= \mathcal{N}[\mu_2^\prime(x_1), \sigma_2^\prime](x_2),
\end{align*}
$$
donde la media es
$$
\mu_2^\prime(x_1) = \mu_2 + \rho \frac{\sigma_2}{\sigma_1} (x_1 - \mu_1),
$$
y la varianza
$$
{\sigma_2^\prime}^2 = \sigma_2^2 (1 - \rho^2).
$$

* Definimos las funciones para calcular la media `get_mu_2_prime` y la desviación estándar `get_sigma_2_prime`.
* Usando `np.random.normal`, generamos primero `x_1` y luego `x_2`, usando las medias y las varianzas que correspondan.
* Repetimos la simulación `n = 1000` veces.
* Guardamos los resultados usando las variables `x1, x2 = get_x1_x2(mu1=mu1, mu2=mu2, sigma1=sigma1, sigma2=sigma2, rho=rho, n=n)`.

Un punto $\mathbf{x}=(x_1, x_2)$ pertenece al interior de la elipse $r\sigma$, si la forma cuadrática $q(\mathbf{x})$ satisface que
$$
\begin{align*}
q(\mathbf{x}) &= (\mathbf{x}-\boldsymbol{\mu}_X)^T \Sigma_X^{-1} (\mathbf{x}-\boldsymbol{\mu}_X)\\
&\le r^2.
\end{align*}
$$

* Calculamos la fracción de eventos que caen dentro de la elipse $r\sigma$ como aquellos que satisfagan $q(\mathbf{x})\le r^2$.
* Para ello, utilizamos `matplotlib.path.Path` y su método `contains_points` para contar cuántos puntos caen dentro de la elipse y guardamos el resultado en la variable `probabilidad_aproximada`.
* Calculamos la probabilidad exacta como la acumulada `probabilidad_exacta = (1 - np.exp(-0.5 * r ** 2)) * 100.0`.

Para producir una elipse $r\sigma$, asociada a una variable binormal, podemos transformar un cículo unitario centrado en el origen [[Wikipedia - Code sample](https://en.wikipedia.org/wiki/File:MultivariateNormal.png#Summary)].

Este círculo unitario corresponde a la superficie $1\sigma$ de una varible binormal estándar.

Podemos re-escalear el círculo unitario, multiplicando por $r$.

Luego, transformarlo con la matriz triangular inferior de la descomposición de Cholesky de $\Sigma_X$ [[Use the Cholesky transformation to correlate and uncorrelate variables](https://blogs.sas.com/content/iml/2012/02/08/use-the-cholesky-transformation-to-correlate-and-uncorrelate-variables.html)].

Y, finalmente, trasladarlo sumando la media $\boldsymbol{\mu}_X$.

* Usando `t = np.linspace(0, 2 * np.pi, 1000)`, parametrizamos un círculo unitario `circulo = np.column_stack([np.cos(t), np.sin(t)])`.
* Descomponemos `Sigma_X` con `np.linalg.cholesky` y obtenemos la matriz `L`.
* Transformamos el círculo para obtener la elipse $r\sigma$, como `elipse = r * circulo @ L.T + mu_X`.

## Observación
La descomposición de Cholesky consiste en expresar una matriz hermítica y definida positiva, lo cual es satisfecho por una matriz de covarianza $\Sigma$, como
$$
\Sigma = LL^T.
$$

Si transformamos una variable aleatoria bidimensional $Z$, con media cero y matriz de covarianza identidad, es decir
$$
\boldsymbol{\mu}_Z = \mathbf{0},
$$
y
$$
\Sigma_Z = \mathbb{1},
$$
multiplicándola por la matriz $L$, es decir,
$$
\tilde Z = L Z
$$
obtenemos otra variable aleatoria $\tilde Z$ con media cero, pero con covarianza $\Sigma$, pues
$$
\begin{align*}
\boldsymbol{\mu}_{\tilde Z} &= \mathrm{E}(\tilde Z)\\
&= \mathrm{E}(LZ)\\
&= L\underbrace{\mathrm{E}(Z)}_{\mathbf{0}}\\
&= \mathbf{0},
\end{align*}
$$
y
$$
\begin{align*}
\Sigma_{\tilde Z} &= \mathrm{E}({\tilde Z\tilde Z}^T)\\
&= \mathrm{E}(LZZ^T L^T)\\
&= L\underbrace{\mathrm{E}(ZZ^T)}_{\mathbb{1}}L^T\\
&= LL^T\\
&= \Sigma.
\end{align*}
$$

Usando esta observación, tratamos de justificar la intuición de transformar un círculo de radio $r$, que representa la región $r\sigma$ de una variable binormal estándar, en una elipse $r\sigma$ de una variable binormal con la covarianza $\Sigma$ deseada. Y esto se consigue aplicando la matriz $L$.

In [2]:
"""Ejercicio 7 (Para entregar)"""
import numpy as np
from matplotlib.path import Path
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

np.random.seed(42)


def get_mu_X(mu1, mu2):
    """Retorna vector de medias."""
    return np.array([mu1, mu2])


def get_Sigma_X(sigma1, sigma2, rho):
    """Retorna la matriz de covarianza."""
    return np.array(
        [[sigma1 ** 2, rho * sigma1 * sigma2], [rho * sigma1 * sigma2, sigma2 ** 2]]
    )


def get_mu_2_prime(x1, mu1, mu2, sigma1, sigma2, rho):
    """Retorna la media de la distribucion condicional."""
    return mu2 + rho * (x1 - mu1) * sigma2 / sigma1


def get_sigma_2_prime(sigma2, rho):
    """Retorna la varianza de la distribucion condicional."""
    return np.sqrt((1 - rho ** 2) * sigma2 ** 2)


def get_x1_x2(mu1, mu2, sigma1, sigma2, rho, n):
    """Retorna variable aleatoria binormal (x1, x2)."""
    x1 = np.random.normal(size=n, loc=mu1, scale=sigma1)
    x2 = np.random.normal(
        loc=get_mu_2_prime(
            x1=x1, mu1=mu1, mu2=mu2, sigma1=sigma1, sigma2=sigma2, rho=rho
        ),
        scale=get_sigma_2_prime(sigma2=sigma2, rho=rho),
    )
    return x1, x2


def plot_ej7(mu1, mu2, sigma1, sigma2, rho, n, r):
    """Grafica Ejercicio 7."""
    x1, x2 = get_x1_x2(mu1=mu1, mu2=mu2, sigma1=sigma1, sigma2=sigma2, rho=rho, n=n)
    mu_X = get_mu_X(mu1=mu1, mu2=mu2)
    Sigma_X = get_Sigma_X(sigma1=sigma1, sigma2=sigma2, rho=rho)
    L = np.linalg.cholesky(Sigma_X)
    t = np.linspace(0, 2 * np.pi, 1000)
    circulo = np.column_stack([np.cos(t), np.sin(t)])
    elipse = r * circulo @ L.T + mu_X
    probabilidad_aproximada = (
        Path(elipse).contains_points(np.column_stack([x1, x2])).sum() / n * 100.0
    )
    probabilidad_exacta = (1 - np.exp(-0.5 * r ** 2)) * 100.0
    sns.set_context("paper", font_scale=1.5)
    g = sns.jointplot(x=x1, y=x2, alpha=5 / np.sqrt(n), height=8)
    ax = g.ax_joint
    ax.plot(*elipse.T, alpha=0.9, c="k")
    text = (
        rf"Elipse                        {r}$\sigma$"
        "\n"
        f"Prob. aproximada {probabilidad_aproximada:.2f}%"
        "\n"
        f"Prob. exacta         {probabilidad_exacta:.2f}%"
    )
    ax.text(x=0.05, y=1.0, s=text, ha="left", va="top", transform=ax.transAxes)
    ax.set_xlim(-1, 5.5)
    ax.set_ylim(-1.75, 4.75)
    ax.set_xlabel(r"$x_1$")
    ax.set_ylabel(r"$x_2$")
    plt.show()
    plt.close()


def ej7():
    """Ejercicio 7."""
    mu1 = widgets.FloatSlider(description="mu1", value=2.3, min=-0.5, max=5.0)
    mu2 = widgets.FloatSlider(description="mu2", value=1.5, min=-0.5, max=5.0)
    sigma1 = widgets.FloatSlider(description="sigma1", value=1.2, min=0.1, max=2.0)
    sigma2 = widgets.FloatSlider(description="sigma2", value=0.5, min=0.1, max=2.0)
    rho = widgets.FloatSlider(description="rho", value=0.7, min=-0.9, max=0.9)
    n = widgets.IntSlider(description="n", value=1000, min=1000, max=10000, step=1000)
    r = widgets.FloatSlider(description="r", value=1.0, min=0.5, max=4.0, step=0.5)
    out = widgets.interactive_output(
        plot_ej7,
        {
            "mu1": mu1,
            "mu2": mu2,
            "sigma1": sigma1,
            "sigma2": sigma2,
            "rho": rho,
            "n": n,
            "r": r,
        },
    )
    title = widgets.Label(
        "Seleccionar parámetros",
        layout=widgets.Layout(display="flex", justify_content="center"),
    )
    sliders = [title, mu1, mu2, sigma1, sigma2, rho, n, r]
    display(
        widgets.HBox(
            [out, widgets.VBox(sliders)],
            layout=widgets.Layout(width="100%", display="flex", align_items="center"),
        )
    )


if __name__ == "__main__":
    ej7()
